In [1]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
openai_api_key = os.environ["OPENAI_API_KEY"]

## Basic App: Question & Answering from a Document

In [2]:
from langchain_openai import OpenAI

In [3]:
llm = OpenAI()

**Load the text file**

In [4]:
from langchain.document_loaders import TextLoader

In [5]:
loader = TextLoader("data/be-good.txt")

In [6]:
document = loader.load()

**The document is loaded as a Python list with metadata**

In [7]:
print(type(document))

<class 'list'>


In [8]:
print(len(document))

1


In [9]:
print(document[0].metadata)

{'source': 'data/be-good.txt'}


In [10]:
print(f"You have {len(document)} document.")

You have 1 document.


In [11]:
print(f"Your document has {len(document[0].page_content)} characters")

Your document has 16706 characters


**Split the document in small chunks**

In [12]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [13]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=3000,
    chunk_overlap=400
)

In [14]:
document_chunks = text_splitter.split_documents(document)

In [15]:
print(f"Now you have {len(document_chunks)} chunks.")

Now you have 8 chunks.


**Convert text chunks in numeric vectors (called "embeddings")**

In [16]:
from langchain.embeddings.openai import OpenAIEmbeddings

In [17]:
embeddings = OpenAIEmbeddings()

/Users/juliocolomer/.pyenv/versions/3.11.4/envs/venv020124/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.1.0 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


**Load the embeddings to a vector database**

In [18]:
from langchain.vectorstores import FAISS

*Careful: the next operation is expensive in OpenAI*

In [19]:
stored_embeddings = FAISS.from_documents(document_chunks, embeddings)

**Create a Retrieval Question & Answering Chain**

In [20]:
from langchain.chains import RetrievalQA

In [21]:
QA_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=stored_embeddings.as_retriever()
)

**Now we have a Question & Answering APP**

In [22]:
question = """
What is this article about? 
Describe it in less than 100 words.
"""

In [23]:
QA_chain.run(question)

/Users/juliocolomer/.pyenv/versions/3.11.4/envs/venv020124/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


'\n\nThis article is about the importance of being good in business, both as a moral principle and as a strategic advantage. The author argues that companies that prioritize making something people want and being good to their users over making money often end up being more successful in the long run. The article also discusses various examples, including Craigslist and Google, to support this idea. The author also suggests that being good can act as a compass for making decisions and can attract top talent.'

In [24]:
question2 = """
And how does it explain how to create somethin people want?
"""

In [25]:
QA_chain.run(question2)

' It explains that creating something people want can be achieved by following the motto of "Make something people want" and not worrying too much about the business model at first. By focusing on creating something that truly benefits and helps people, it can lead to a successful and desirable product. Additionally, being good and benevolent can attract investors, customers, and employees, and can act as a guiding compass for decision making.'

## Simple Agent

In [26]:
#!pip install numexpr

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.8/91.8 kB 1.0 MB/s eta 0:00:00ta 0:00:01

[notice] A new release of pip is available: 23.1.2 -> 23.3.2
[notice] To update, run: python3.11 -m pip install --upgrade pip


In [27]:
from langchain.agents import load_tools
from langchain.llms import OpenAI
from langchain.agents import AgentType

llm=OpenAI()

tool_names = ["llm-math"]
tools = load_tools(tool_names, llm=llm)
tools

/Users/juliocolomer/.pyenv/versions/3.11.4/envs/venv020124/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.llms.openai.OpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  warn_deprecated(


[Tool(name='Calculator', description='Useful for when you need to answer questions about math.', func=<bound method Chain.run of LLMMathChain(llm_chain=LLMChain(prompt=PromptTemplate(input_variables=['question'], template='Translate a math problem into a expression that can be executed using Python\'s numexpr library. Use the output of running this code to answer the question.\n\nQuestion: ${{Question with math problem.}}\n```text\n${{single line mathematical expression that solves the problem}}\n```\n...numexpr.evaluate(text)...\n```output\n${{Output of running the code}}\n```\nAnswer: ${{Answer}}\n\nBegin.\n\nQuestion: What is 37593 * 67?\n```text\n37593 * 67\n```\n...numexpr.evaluate("37593 * 67")...\n```output\n2518731\n```\nAnswer: 2518731\n\nQuestion: 37593^(1/5)\n```text\n37593**(1/5)\n```\n...numexpr.evaluate("37593**(1/5)")...\n```output\n8.222831614237718\n```\nAnswer: 8.222831614237718\n\nQuestion: {question}\n'), llm=OpenAI(client=<openai.resources.completions.Completions o

In [28]:
from langchain.agents import initialize_agent

agent = initialize_agent(tools,
                         llm,
                         agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
                         verbose=True,
                         max_iterations=3)

/Users/juliocolomer/.pyenv/versions/3.11.4/envs/venv020124/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  warn_deprecated(


In [29]:
agent.run("What is 133 by 142?")



> Entering new AgentExecutor chain...
 I should use a calculator to solve this problem.
Action: Calculator
Action Input: 133 * 142
Observation: Answer: 18886
Thought: I now know the final answer.
Final Answer: 18886

> Finished chain.


'18886'

**Let's make the agent fail**

In [30]:
agent.run("Who was the wife of Napoleon Bonaparte?")



> Entering new AgentExecutor chain...
 I do not have access to the internet, so I cannot look up the answer.
Action: I can use my knowledge of history to try to answer this question.
Action Input: Napoleon Bonaparte
Observation: I can use my knowledge of history to try to answer this question. is not a valid tool, try one of [Calculator].
Thought: I do not have access to the internet, so I cannot look up the answer.
Action: I can use my knowledge of history to try to answer this question.
Action Input: Napoleon Bonaparte
Observation: I can use my knowledge of history to try to answer this question. is not a valid tool, try one of [Calculator].
Thought: I can use my knowledge of history to try to answer this question.
Action: I can use the Calculator to perform a quick search.
Action Input: Napoleon Bonaparte wife
Observation: I can use the Calculator to perform a quick search. is not a valid tool, try one of [Calculator].
Thought:

> Finished chain.


'Agent stopped due to iteration limit or time limit.'

## Custom Agent

In [31]:
from typing import Optional
from langchain.tools import BaseTool
from langchain.callbacks.manager import AsyncCallbackManagerForToolRun
from langchain.callbacks.manager import CallbackManagerForToolRun

In [32]:
class CustomSearchTool(BaseTool):
    name = "article search"
    description = "useful for when you need to answer questions about our article"

    def _run(self, query: str, run_manager: Optional[CallbackManagerForToolRun] = None) -> str:
        store = stored_embeddings.as_retriever()
        docs = store.get_relevant_documents(query)
        text_list = [doc.page_content for doc in docs]
        return "\n".join(text_list)

    async def _arun(self, query: str, run_manager: Optional[AsyncCallbackManagerForToolRun] = None) -> str:
        """Use the tool asynchronously."""
        raise NotImplementedError("custom_search does not support async")

In [33]:
from langchain.agents import AgentType

tools = [CustomSearchTool()]

agent = initialize_agent(
    tools, 
    llm, 
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, 
    verbose=True, 
    max_iterations=3
)

In [34]:
agent.run("What is this article about? Describe it in less than 100 words.")



> Entering new AgentExecutor chain...
 I should use the article search tool to find the article and read the abstract to get a general understanding of the topic.
Action: article search
Action Input: "article about"
Observation: Be good
April 2008(This essay is derived from a talk at the 2008 Startup School.)About a month after we started Y Combinator we came up with the
phrase that became our motto: Make something people want.  We've
learned a lot since then, but if I were choosing now that's still
the one I'd pick.Another thing we tell founders is not to worry too much about the
business model, at least at first.  Not because making money is
unimportant, but because it's so much easier than building something
great.A couple weeks ago I realized that if you put those two ideas
together, you get something surprising.  Make something people want.
Don't worry too much about making money.  What you've got is a
description of a charity.When you get an unexpected result like this, it coul

BadRequestError: Error code: 400 - {'error': {'message': "This model's maximum context length is 4097 tokens, however you requested 4979 tokens (4723 in your prompt; 256 for the completion). Please reduce your prompt; or completion length.", 'type': 'invalid_request_error', 'param': None, 'code': None}}